In [4]:
!pip install praw pandas python-dotenv


## Fetching Reddit Data via API

This section demonstrates how we collected data from Reddit programmatically using the **PRAW (Python Reddit API Wrapper)** library. The process involves connecting to the Reddit API, fetching posts from selected subreddits, and storing the data in a structured format.

**Steps:**

1. **API Connection:**  
   - Load Reddit API credentials from environment variables (`CLIENT_ID`, `CLIENT_SECRET`, `USER_AGENT`).  
   - Establish a read-only connection to Reddit using `praw.Reddit()`.

2. **Subreddit Selection:**  
   - The script targets multiple subreddits: `r/technology`, `r/mentalhealth`, and `r/AskReddit`.  
   - For each subreddit, it fetches **20 posts**, ignoring stickied posts.

3. **Data Collection:**  
   For each post, the following information is extracted:
   - **Title**
   - **Body**
   - **Author**
   - **Number of upvotes**
   - **Number of comments**
   - **Timestamp (UTC)**

4. **Data Storage and Export:**  
   - All collected posts are stored in a **Pandas DataFrame** for easy manipulation.  
   - Data is sorted by subreddit and upvotes for a cleaner preview.  
   - The raw data is exported to `raw_reddit_data.csv`.


In [41]:
import logging
logging.getLogger("praw").setLevel(logging.ERROR)
import praw
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime, timezone
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load environment variables
load_dotenv()
CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
USER_AGENT = os.getenv("REDDIT_USER_AGENT")

# Basic check for credentials
if not CLIENT_ID or not CLIENT_SECRET or not USER_AGENT:
    raise ValueError("Reddit API credentials are missing in the environment variables.")

print("="*70)
print(" Connecting to Reddit API...")
reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=CLIENT_SECRET,
                     user_agent=USER_AGENT)
reddit.read_only = True
print(" Connected successfully!")
print("="*70)

# Subreddits & posts config
subreddits = ["technology", "mentalhealth", "AskReddit"]
posts_per_subreddit = 20
all_posts = []

# Fetch posts
for sub in subreddits:
    print(f"\n Fetching {posts_per_subreddit} posts from r/{sub}...")
    subreddit_posts = []

    for post in reddit.subreddit(sub).hot(limit=posts_per_subreddit + 5):  # extra in case of stickied
        if post.stickied:
            continue

        author_name = str(post.author) if post.author else "[deleted]"
        body_text = post.selftext if post.selftext else ""

        # Collect post
        post_data = {
            "subreddit": sub,
            "title": post.title,
            "body": body_text,
            "author": author_name,
            "upvotes": post.score,
            "num_comments": post.num_comments,
            "timestamp": datetime.fromtimestamp(post.created_utc, tz=timezone.utc)
        }
        subreddit_posts.append(post_data)

        # Print all posts (or first 20)
        if len(subreddit_posts) <= posts_per_subreddit:
            print(f"\n POST #{len(subreddit_posts)}")
            print(f"Title: {post.title}")
            print(f"Author: {author_name}")
            print(f"Upvotes: {post.score} | Comments: {post.num_comments}")
            print(f"Timestamp (UTC): {post_data['timestamp']}")
            print(f"Body Preview: {body_text[:150].replace(chr(10),' ')}{'...' if len(body_text) > 150 else ''}")

        # Stop when we have enough posts
        if len(subreddit_posts) >= posts_per_subreddit:
            break

    all_posts.extend(subreddit_posts)
    print(f" Collected {len(subreddit_posts)} posts from r/{sub}")

# Convert to DataFrame
df = pd.DataFrame(all_posts, columns=[
    "subreddit","title","body","author","upvotes","num_comments","timestamp"
])

# sort by subreddit then upvotes descending for better preview
df.sort_values(by=["subreddit","upvotes"], ascending=[True, False], inplace=True)

print("\n" + "="*70)
print(" Raw Reddit Data Preview (First 10 rows) ")
print("="*70)
print(df.head(10).to_string(index=False))

# Save CSV
csv_filename = "raw_reddit_data.csv"
df.to_csv(csv_filename, index=False, encoding="utf-8")
print(f"\n Saved {csv_filename} successfully!")


 Connecting to Reddit API...
 Connected successfully!

 Fetching 20 posts from r/technology...

 POST #1
Title: IBM CEO says there is 'no way' spending trillions on AI data centers will pay off at today's infrastructure costs
Author: captain-price-
Upvotes: 17768 | Comments: 1465
Timestamp (UTC): 2025-12-02 15:04:53+00:00
Body Preview: 

 POST #2
Title: More than 1,000 Amazon employees sign open letter warning the company’s AI ‘will do staggering damage to democracy, our jobs, and the earth’
Author: Moonskaraos
Upvotes: 1351 | Comments: 43
Timestamp (UTC): 2025-12-02 18:19:53+00:00
Body Preview: 

 POST #3
Title: Zig quits GitHub, says Microsoft's AI obsession has ruined the service
Author: rkhunter_
Upvotes: 538 | Comments: 14
Timestamp (UTC): 2025-12-02 19:24:22+00:00
Body Preview: 

 POST #4
Title: The rise of deepfake pornography in schools: ‘One girl was so horrified she vomited’ | The use of ‘nudify’ apps is becoming more and more prevalent, with hundreds of teachers having seen 

#  Text Cleaning & Preprocessing  


This section explains the preprocessing workflow applied to clean and prepare raw Reddit text data for NLP tasks like classification and sentiment analysis.

---

## 📌 **Overview**
Reddit posts often contain noise such as URLs, punctuation, numbers, and unnecessary words. To improve text quality and make it useful for NLP models, a structured preprocessing pipeline was applied.

---

## 🔄 **Preprocessing Steps**

### **1️⃣ Lowercasing**
All text is converted to lowercase so words like "Food" and "food" are treated the same.

### **2️⃣ Removing Noise**
Using Python's `re` library, the following elements are removed:
- URLs  
- Mentions (`@username`)  
- Hashtags (`#topic`)  
- Punctuation  
- Numbers  
- Special characters  

### **3️⃣ Tokenization**
Each sentence is split into individual words using **NLTK’s tokenizer**, preparing it for stopword removal and analysis.

### **4️⃣ Stopword Removal**
Stopwords are removed using:
- **NLTK English stopword list**
- **Custom Reddit-specific stopwords** such as:  
  `like, want, think, people, get, good, many, even`

### **5️⃣ Whitespace Normalization**
Extra spaces, multiple spaces, and line breaks are cleaned for consistent formatting.

---

## 📘 **Libraries Used**
- **NLTK** – tokenization, stopword removal  
- **re** – regex-based text cleaning  
- **pandas** – dataset handling & creation of new columns  
- **tabulate** – printing before/after comparison tables  

---

## 📄 **New Columns Generated**
| Column Name | Description |
|-------------|-------------|
| **cleaned_title** | Cleaned version of the post title |
| **cleaned_body** | Cleaned version of the post body |
| **full_text** | Combined original title and body |
| **cleaned_full_text** | Fully cleaned version of full_text |

---

## 💾 **Final Output**
The processed dataset is saved as:
preprocessed_reddit_data.csv


In [50]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import warnings
from tabulate import tabulate

warnings.filterwarnings('ignore')

print("="*70)
print(" TEXT CLEANING & PREPROCESSING")
print("="*70)

print(" Downloading NLTK resources...")
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
print(" NLTK resources ready")

print("\n Loading data from raw_reddit_data.csv...")
df = pd.read_csv('raw_reddit_data.csv')
print(f" Loaded {len(df)} posts")

# Define stopwords
stop_words = set(stopwords.words('english'))
reddit_stopwords = [
    'like', 'get', 'would', 'could', 'also', 'really', 'one', 'much', 'many',
    'even', 'still', 'maybe', 'think', 'know', 'said', 'people', 'thing', 'things',
    'go', 'see', 'want', 'need', 'make', 'take', 'good'
]
stop_words.update(reddit_stopwords)

def preprocess_text(text, remove_stopwords=True, remove_numbers=True):
    if not isinstance(text, str) or pd.isna(text):
        return ""

    text = text.lower().strip()  # lowercase and trim
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # remove URLs
    text = re.sub(r'@\w+|#\w+', '', text)  # remove mentions & hashtags
    text = re.sub(r'[^a-zA-Z\s]' if remove_numbers else r'[^\w\s]', '', text)  # remove punctuation/numbers
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces
    tokens = word_tokenize(text)

    if remove_stopwords:
        tokens = [w for w in tokens if w not in stop_words]

    return ' '.join(tokens)

print("\n Applying preprocessing to posts...")
df['cleaned_title'] = df['title'].apply(preprocess_text)
df['cleaned_body'] = df['body'].apply(lambda x: preprocess_text(x) if pd.notna(x) else "")
df['full_text'] = df['title'] + ' ' + df['body'].fillna('')
df['cleaned_full_text'] = df['full_text'].apply(preprocess_text)

print(" Preprocessing complete!")
print(f"   • Original columns: {len([c for c in df.columns if 'cleaned' not in c])}")
print(f"   • New columns: cleaned_title, cleaned_body, full_text, cleaned_full_text")

sample_indices = []
for sub in df['subreddit'].unique():
    sub_df = df[df['subreddit'] == sub]
    if len(sub_df) > 0:
        sample_indices.append(sub_df.index[0])

# Ensure exactly 5 samples
if len(sample_indices) < 5:
    remaining = df.index.difference(sample_indices)[:5 - len(sample_indices)]
    sample_indices.extend(remaining)
sample_indices = sample_indices[:5]

truncate = lambda text, n: text[:n] + ("..." if len(text) > n else "")

# Prepare comparison data
comparison_data = []
for idx in sample_indices:
    original_title = df.loc[idx, 'title']
    cleaned_title = df.loc[idx, 'cleaned_title']
    body_text = df.loc[idx, 'body']
    original_body = "" if pd.isna(body_text) else str(body_text)
    cleaned_body = df.loc[idx, 'cleaned_body']
    subreddit = df.loc[idx, 'subreddit']

    orig_words = len(original_title.split()) + len(original_body.split())
    clean_words = len(cleaned_title.split()) + len(cleaned_body.split())
    reduction = orig_words - clean_words
    reduction_pct = (reduction / orig_words * 100) if orig_words > 0 else 0

    comparison_data.append({
        '#': idx + 1,
        'Subreddit': f"r/{subreddit}",
        'Original Title': truncate(original_title, 60),
        'Cleaned Title': truncate(cleaned_title, 60),
        'Original Body': truncate(original_body, 80),
        'Cleaned Body': truncate(cleaned_body, 80),
        'Word Count': f"{orig_words} → {clean_words}",
        'Reduction': f"{reduction} ({reduction_pct:.1f}%)"
    })

comparison_df = pd.DataFrame(comparison_data)

print("\n BEFORE/AFTER COMPARISON (5 Samples)")
print("="*120)
print(tabulate(comparison_df, headers='keys', tablefmt='fancy_grid', showindex=False, stralign='left'))
print("="*120)

print("\n BODY TEXT COMPARISON:")
for _, row in comparison_df.iterrows():
    print(f"\n{'='*100}")
    print(f"Sample {row['#']} ({row['Subreddit']}) | Words: {row['Word Count']} | Reduction: {row['Reduction']}")
    print(f"Original: {row['Original Body']}")
    print(f"Cleaned:  {row['Cleaned Body']}")

print("\n" + "="*70)
print(" PREPROCESSING STEPS DEMONSTRATION")
print("="*70)

sample_idx = sample_indices[0]
sample_text = df.loc[sample_idx, 'title']
print(f"\n Sample Text: '{sample_text}'")

step_lower = sample_text.lower()
step_no_url = re.sub(r'http\S+|www\S+|https\S+', '', step_lower)
step_no_punct = re.sub(r'[^a-zA-Z\s]', '', step_no_url)
tokens = word_tokenize(step_no_punct)
tokens_no_stop = [w for w in tokens if w not in stop_words]

steps = [
    ("1. Lowercase", step_lower),
    ("2. Remove URLs", step_no_url),
    ("3. Remove punctuation", step_no_punct),
    ("4. Tokenize", tokens),
    ("5. Remove stopwords", tokens_no_stop),
    ("6. Final cleaned", preprocess_text(sample_text))
]

for name, result in steps:
    if isinstance(result, list):
        print(f"{name}: {result[:10]}...")
    else:
        print(f"{name}: '{result[:80]}'")

print("\n" + "="*70)
print(" SAVING PROCESSED DATA")
print("="*70)

output_file = "preprocessed_reddit_data.csv"
df.to_csv(output_file, index=False, encoding='utf-8')
print(f" Saved to: {output_file}")
print(f" Total posts: {len(df)}")
print(f" File size: {df.memory_usage(deep=True).sum() / 1024:.1f} KB")


 TEXT CLEANING & PREPROCESSING
 NLTK resources ready

 Loading data from raw_reddit_data.csv...
 Loaded 60 posts

 Applying preprocessing to posts...
 Preprocessing complete!
   • Original columns: 8
   • New columns: cleaned_title, cleaned_body, full_text, cleaned_full_text

 BEFORE/AFTER COMPARISON (5 Samples)
╒═════╤════════════════╤═════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════╤══════════════╤═════════════╕
│   # │ Subreddit      │ Original Title                                                  │ Cleaned Title                                                   │ Original Body                                                                       │ Cleaned Body                                                                        │

## Part 3: Post Classification

- Load preprocessed Reddit data
- Perform sentiment analysis using **VADER** and **TextBlob**
- Combine results to get a final sentiment
- Classify posts into topics based on keywords

---

## Sentiment Analysis

Sentiments analyzed:

- **Positive** 😊
- **Neutral** 😐
- **Negative** 😔

In [51]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon', quiet=True)

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print("="*70)
print(" PART 3: POST CLASSIFICATION")
print("="*70)

# Load data
df = pd.read_csv('preprocessed_reddit_data.csv')
print(f" Loaded {len(df)} preprocessed posts")

# Initialize analyzer
vader = SentimentIntensityAnalyzer()

def analyze_sentiment_vader(text):
    """Analyze sentiment using VADER"""
    if not isinstance(text, str) or not text.strip():
        return {'sentiment': 'neutral', 'compound': 0}

    scores = vader.polarity_scores(text)
    compound = scores['compound']

    if compound >= 0.05:
        sentiment = 'positive'
    elif compound <= -0.05:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'

    return {'sentiment': sentiment, 'compound': compound}

def analyze_sentiment_textblob(text):
    """Analyze sentiment using TextBlob"""
    if not isinstance(text, str) or not text.strip():
        return {'sentiment': 'neutral', 'polarity': 0}

    blob = TextBlob(text)
    polarity = blob.sentiment.polarity

    if polarity > 0:
        sentiment = 'positive'
    elif polarity < 0:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'

    return {'sentiment': sentiment, 'polarity': polarity}

def get_final_sentiment(text):
    """Combine VADER and TextBlob results"""
    vader_result = analyze_sentiment_vader(text)
    textblob_result = analyze_sentiment_textblob(text)

    # If both agree, use that
    if vader_result['sentiment'] == textblob_result['sentiment']:
        return vader_result['sentiment']

    # If one is neutral, prefer the non-neutral
    if vader_result['sentiment'] == 'neutral':
        return textblob_result['sentiment']
    if textblob_result['sentiment'] == 'neutral':
        return vader_result['sentiment']

    # Otherwise use VADER (better for social media)
    return vader_result['sentiment']

print(" Performing sentiment analysis...")

# Get individual model results
df['vader_sentiment'] = df['cleaned_full_text'].apply(lambda x: analyze_sentiment_vader(x)['sentiment'])
df['vader_compound'] = df['cleaned_full_text'].apply(lambda x: analyze_sentiment_vader(x)['compound'])
df['textblob_sentiment'] = df['cleaned_full_text'].apply(lambda x: analyze_sentiment_textblob(x)['sentiment'])
df['textblob_polarity'] = df['cleaned_full_text'].apply(lambda x: analyze_sentiment_textblob(x)['polarity'])

# Get final combined sentiment
df['final_sentiment'] = df['cleaned_full_text'].apply(get_final_sentiment)

print(" Sentiment analysis complete!")

print(" Performing topic classification...")

topic_keywords = {
    'technology': ['tech', 'ai', 'software', 'computer', 'app', 'internet', 'digital',
                   'data', 'code', 'programming', 'machine', 'algorithm', 'cyber',
                   'security', 'cloud', 'crypto', 'phone', 'website', 'windows', 'android'],
    'mental_health': ['mental', 'therapy', 'anxiety', 'stress', 'depression', 'health',
                     'wellness', 'mindfulness', 'psychology', 'counseling', 'emotion',
                     'mind', 'happy', 'sad', 'angry', 'treatment', 'support', 'trauma',
                     'ptsd', 'disorder'],
    'education': ['school', 'college', 'university', 'study', 'learn', 'education',
                 'course', 'student', 'teacher', 'knowledge', 'skill', 'degree',
                 'exam', 'homework', 'class', 'professor', 'tuition', 'campus'],
    'entertainment': ['movie', 'music', 'game', 'tv', 'show', 'entertainment', 'fun',
                     'comedy', 'drama', 'film', 'song', 'artist', 'actor', 'celebrity',
                     'series', 'episode', 'season', 'netflix', 'youtube', 'spotify'],
    'politics': ['government', 'politic', 'election', 'vote', 'law', 'policy',
                'democrat', 'republican', 'congress', 'senate', 'president',
                'bill', 'senator', 'representative', 'supreme', 'court']
}

def classify_topic(text):
    """Classify text into topics based on keyword matching"""
    if not isinstance(text, str):
        return 'other'

    text_lower = text.lower()
    scores = {}

    for topic, keywords in topic_keywords.items():
        score = sum(1 for keyword in keywords if keyword in text_lower)
        scores[topic] = score

    if max(scores.values()) > 0:
        return max(scores, key=scores.get)
    else:
        return 'other'

# Apply topic classification
df['primary_topic'] = df['cleaned_full_text'].apply(classify_topic)

print(" Topic classification complete!")


print("\n" + "="*70)
print(" CONFUSION MATRIX & MODEL AGREEMENT")
print("="*70)

# Calculate accuracy and confusion matrix
accuracy = accuracy_score(df['vader_sentiment'], df['textblob_sentiment'])
cm = confusion_matrix(df['vader_sentiment'], df['textblob_sentiment'],
                      labels=['negative', 'neutral', 'positive'])

print(f"\n Model Agreement Analysis:")
print(f"   • VADER vs TextBlob Accuracy: {accuracy:.2%}")
print(f"   • Total Posts: {len(df)}")
print(f"   • Agreements: {int(accuracy * len(df))}")
print(f"   • Disagreements: {int((1 - accuracy) * len(df))}")

print("\n Confusion Matrix (VADER vs TextBlob):")
print(" " * 15 + "TextBlob Predictions")
print(" " * 15 + "-" * 40)
print(" " * 10 + "Negative   Neutral   Positive")
print("-" * 45)

labels = ['negative', 'neutral', 'positive']
for i, true_label in enumerate(labels):
    row = f"VADER {true_label:<7}"
    for j in range(3):
        row += f"{cm[i][j]:^10}"
    print(row)

print("\n" + "="*70)
print(" CLASSIFICATION RESULTS")
print("="*70)

# Sentiment Distribution
print("\n SENTIMENT DISTRIBUTION:")
print("-" * 40)

sentiment_counts = df['final_sentiment'].value_counts()
total = len(df)

print(f"{'Sentiment':<10} {'Count':<8} {'Percentage':<12} {'Emoji':<5}")
print("-" * 40)

emoji_map = {'positive': '😊', 'neutral': '😐', 'negative': '😔'}
for sentiment, count in sentiment_counts.items():
    percentage = (count / total) * 100
    emoji = emoji_map.get(sentiment, '❓')
    print(f"{sentiment:<10} {count:<8} {percentage:<11.1f}% {emoji:<5}")

# Topic Distribution
print("\n TOPIC DISTRIBUTION:")
print("-" * 40)

topic_counts = df['primary_topic'].value_counts()

print(f"{'Topic':<15} {'Count':<8} {'Percentage':<12}")
print("-" * 40)

for topic, count in topic_counts.items():
    percentage = (count / total) * 100
    print(f"{topic:<15} {count:<8} {percentage:<11.1f}%")

# Sentiment by Subreddit
print("\n SENTIMENT BY SUBREDDIT:")
print("-" * 50)

print(f"{'Subreddit':<15} {'Positive':<10} {'Neutral':<10} {'Negative':<10} {'Dominant':<10}")
print("-" * 50)

for subreddit in df['subreddit'].unique():
    sub_df = df[df['subreddit'] == subreddit]
    positive = len(sub_df[sub_df['final_sentiment'] == 'positive'])
    neutral = len(sub_df[sub_df['final_sentiment'] == 'neutral'])
    negative = len(sub_df[sub_df['final_sentiment'] == 'negative'])
    dominant = sub_df['final_sentiment'].value_counts().index[0]

    print(f"r/{subreddit:<13} {positive:<10} {neutral:<10} {negative:<10} {dominant:<10}")

# Topic by Subreddit
print("\n TOPIC BY SUBREDDIT:")
print("-" * 40)

for subreddit in df['subreddit'].unique():
    sub_df = df[df['subreddit'] == subreddit]
    dominant_topic = sub_df['primary_topic'].value_counts().index[0]
    print(f"r/{subreddit}: Most common topic = {dominant_topic}")

print("\n" + "="*70)
print(" SAMPLE CLASSIFIED POSTS ")
print("="*70)

# Select diverse samples
samples = []
for sentiment in ['positive', 'neutral', 'negative']:
    sentiment_samples = df[df['final_sentiment'] == sentiment].head(2).index.tolist()
    samples.extend(sentiment_samples)

samples = samples[:5]

print("\n Sample Classification Results:")
print("-" * 120)

for idx in samples:
    post = df.loc[idx]
    print(f"\n{'='*120}")
    print(f" POST #{idx + 1} | r/{post['subreddit']}")
    print(f"{'='*120}")
    print(f"Title: {post['title'][:80]}...")
    print(f"Cleaned: {post['cleaned_title'][:100]}...")
    print(f"\n Sentiment: {post['final_sentiment']} (VADER: {post['vader_sentiment']}, TextBlob: {post['textblob_sentiment']})")
    print(f" Topic: {post['primary_topic']}")
    print(f" VADER Score: {post['vader_compound']:.3f} | TextBlob Polarity: {post['textblob_polarity']:.3f}")


print("\n" + "="*70)
print(" SAVING FINAL CLASSIFIED DATASET")
print("="*70)

# Reorder columns for better readability
column_order = [
    'subreddit', 'title', 'cleaned_title', 'body', 'cleaned_body',
    'author', 'upvotes', 'num_comments', 'timestamp',
    'vader_sentiment', 'vader_compound',
    'textblob_sentiment', 'textblob_polarity',
    'final_sentiment', 'primary_topic',
    'full_text', 'cleaned_full_text'
]

# Select only existing columns
existing_columns = [col for col in column_order if col in df.columns]
remaining_columns = [col for col in df.columns if col not in existing_columns]
final_columns = existing_columns + remaining_columns

df_final = df[final_columns]

# Save to CSV
output_file = "classified_reddit_data.csv"
df_final.to_csv(output_file, index=False, encoding='utf-8')

print(f" Final dataset saved to: {output_file}")
print(f" File contains: {len(df_final)} posts")
print(f" Total columns: {len(df_final.columns)}")
print(f" File size: {df_final.memory_usage(deep=True).sum() / 1024:.1f} KB")

print(f"\n KEY CLASSIFICATION COLUMNS ADDED:")
classification_cols = ['vader_sentiment', 'textblob_sentiment', 'final_sentiment', 'primary_topic']
for col in classification_cols:
    print(f"   • {col}")

 PART 3: POST CLASSIFICATION
 Loaded 60 preprocessed posts
 Performing sentiment analysis...
 Sentiment analysis complete!
 Performing topic classification...
 Topic classification complete!

 CONFUSION MATRIX & MODEL AGREEMENT

 Model Agreement Analysis:
   • VADER vs TextBlob Accuracy: 66.67%
   • Total Posts: 60
   • Agreements: 40
   • Disagreements: 20

 Confusion Matrix (VADER vs TextBlob):
               TextBlob Predictions
               ----------------------------------------
          Negative   Neutral   Positive
---------------------------------------------
VADER negative    18        9         3     
VADER neutral    2         12        1     
VADER positive    2         3         10    

 CLASSIFICATION RESULTS

 SENTIMENT DISTRIBUTION:
----------------------------------------
Sentiment  Count    Percentage   Emoji
----------------------------------------
negative   32       53.3       % 😔    
positive   16       26.7       % 😊    
neutral    12       20.0       % 😐    

In [53]:
!pip install apscheduler


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 1.2 MB/s eta 0:00:00


# Reddit Real-Time Data Scraper

This project contains a Python script to **collect Reddit posts in real-time** using **PRAW** and save them for analysis. The script runs continuously with a **scheduler** to fetch posts at regular intervals.

---

## Features

- Connects to Reddit using **API credentials** from `.env`  
- Fetches posts from multiple **subreddits** (e.g., `technology`, `mentalhealth`, `AskReddit`)  
- Collects key post information:
  - **Title**
  - **Body**
  - **Author**
  - **Upvotes**
  - **Number of comments**
  - **Timestamp of creation**
  - **Collection timestamp**
- Skips **stickied posts**  
- Handles **errors** gracefully to prevent crashes  
- Combines new and existing data while **removing duplicates**  
- Keeps only the **latest 1000 posts**  
- Saves data to **CSV** (`reddit_live_data.csv`)  
- Prints **summary statistics** including total posts and date range  

---

## Scheduler

- Uses **APScheduler** to run the scraping job automatically every **60 minutes**  
- First run is executed immediately on script start  
- Scheduler continues to run in the **background**, keeping the main thread alive  
- Can be stopped safely using **Ctrl+C**

---

## Dependencies

- **Python 3.x**  
- **PRAW** – Python Reddit API Wrapper  
- **Pandas** – Data handling  
- **APScheduler** – Background scheduling  
- **python-dotenv** – Load environment variables from `.env`  

---

## Usage

1. Create a `.env` file with your Reddit API credentials:
   - `REDDIT_CLIENT_ID`  
   - `REDDIT_SECRET`  
   - `REDDIT_USER_AGENT`  

2. Run the script to **fetch and store Reddit posts continuously**.

3. Access the saved CSV file (`reddit_live_data.csv`) for further **analysis or visualization**.

---

**Note:** The scraper is designed to run safely over long periods, keeping only the **most recent posts** to avoid excessive storage.


In [60]:
from apscheduler.schedulers.background import BackgroundScheduler
import praw
import pandas as pd
from datetime import datetime
import os
import time
from dotenv import load_dotenv

# Load Reddit credentials
load_dotenv()
CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
CLIENT_SECRET = os.getenv("REDDIT_SECRET")
USER_AGENT = os.getenv("REDDIT_USER_AGENT")

reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=CLIENT_SECRET,
                     user_agent=USER_AGENT)
reddit.read_only = True

def fetch_reddit_posts():
    """Fetch and save Reddit posts with error handling"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] Starting data collection...")

    try:
        subreddits = ["technology", "mentalhealth", "AskReddit"]
        posts_per_subreddit = 20
        all_posts = []

        for sub in subreddits:
            try:
                print(f"  → Fetching from r/{sub}")
                for post in reddit.subreddit(sub).hot(limit=posts_per_subreddit + 5):
                    if post.stickied:
                        continue

                    all_posts.append({
                        "subreddit": sub,
                        "title": post.title,
                        "body": post.selftext if post.selftext else "",
                        "author": str(post.author) if post.author else "[deleted]",
                        "upvotes": post.score,
                        "num_comments": post.num_comments,
                        "timestamp": datetime.utcfromtimestamp(post.created_utc),
                        "collected_at": datetime.now()
                    })

            except Exception as sub_error:
                print(f"   Error fetching from r/{sub}: {str(sub_error)}")
                continue

        # Create DataFrame from new posts
        df_new = pd.DataFrame(all_posts)

        output_file = "reddit_live_data.csv"

        # Check if file exists and append data
        if os.path.exists(output_file):
            try:
                existing_df = pd.read_csv(output_file)

                # Convert timestamp strings to datetime
                if 'timestamp' in existing_df.columns:
                    existing_df['timestamp'] = pd.to_datetime(existing_df['timestamp'])

                if 'collected_at' in existing_df.columns:
                    existing_df['collected_at'] = pd.to_datetime(existing_df['collected_at'])

                # Combine old and new data
                df_combined = pd.concat([existing_df, df_new], ignore_index=True)

                # Remove duplicates (based on title + timestamp)
                df_combined = df_combined.drop_duplicates(
                    subset=['title', 'timestamp'],
                    keep='last'
                )

                # Sort by timestamp (descending)
                df_combined = df_combined.sort_values('timestamp', ascending=False)

                # Keep only latest 1000 posts
                if len(df_combined) > 1000:
                    df_combined = df_combined.head(1000)

                # Use combined data
                df_final = df_combined

            except Exception as e:
                print(f"   Error processing existing file, using new data only: {str(e)}")
                df_final = df_new
        else:
            # File doesn't exist, use only new data
            df_final = df_new

        # Save to CSV
        df_final.to_csv(output_file, index=False)

        print(f"   Data saved to {output_file}")
        print(f"   Total posts: {len(df_final)}")

        # Display date range only if we have data
        if len(df_final) > 0 and 'timestamp' in df_final.columns:
            try:
                min_date = df_final['timestamp'].min()
                max_date = df_final['timestamp'].max()
                print(f"   Date range: {min_date} to {max_date}")
            except:
                pass  # Skip if there's an error with date formatting

    except Exception as e:
        print(f"[{datetime.now()}]  Error in fetch_reddit_posts: {str(e)}")

# Simple scheduler setup
scheduler = BackgroundScheduler()

# Schedule the job
scheduler.add_job(
    func=fetch_reddit_posts,
    trigger='interval',
    minutes=60,
    id='reddit_scraper',
    name='Reddit Data Scraper',
    replace_existing=True
)

# Run immediately and start scheduler
print(" Reddit Real-Time Data Scraper")
print("=" * 40)

# First run
fetch_reddit_posts()

# Start scheduler
scheduler.start()
print(" Scheduler started successfully!")

if scheduler.get_jobs():
    print(f" Next run: {scheduler.get_jobs()[0].next_run_time}")

print("\nPress Ctrl+C to stop the scheduler\n")

# Keep main thread alive
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n Stopping scheduler...")+
    scheduler.shutdown(wait=False)
    print(" Done!")

 Reddit Real-Time Data Scraper
[2025-12-02 21:25:16] Starting data collection...
  → Fetching from r/technology
  → Fetching from r/mentalhealth
  → Fetching from r/AskReddit
   Data saved to reddit_live_data.csv
   Total posts: 75
   Date range: 2025-12-01 14:32:16 to 2025-12-02 21:23:15
 Scheduler started successfully!
 Next run: 2025-12-02 22:25:16.873763+00:00

Press Ctrl+C to stop the scheduler


 Stopping scheduler...
 Done!


In [63]:
!pip install pyngrok


In [65]:

!pip install streamlit pandas plotly numpy textblob praw python-dotenv

!python -m textblob.download_corpora

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 111.7 MB/s eta 0:00:00
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [66]:
# Run these commands in your notebook/code cell:

# 1. Install Streamlit
!pip install streamlit

# 2. Install other dependencies
!pip install pandas plotly numpy

# 3. Install TextBlob for sentiment analysis
!pip install textblob

# 4. Install Reddit API wrapper
!pip install praw

# 5. Install environment variable manager
!pip install python-dotenv

# 6. Download TextBlob data
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [67]:
# Run these commands in your notebook/code cell:

# 1. Install Streamlit
!pip install streamlit

# 2. Install other dependencies
!pip install pandas plotly numpy

# 3. Install TextBlob for sentiment analysis
!pip install textblob

# 4. Install Reddit API wrapper
!pip install praw

# 5. Install environment variable manager
!pip install python-dotenv

# 6. Download TextBlob data
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [68]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import os
import json
import numpy as np
from textblob import TextBlob

# Page configuration
st.set_page_config(
    page_title="Reddit Sentiment Dashboard",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for styling
st.markdown("""
<style>
    .main-header {
        font-size: 2.8rem;
        color: #FF5700;
        text-align: center;
        margin-bottom: 1rem;
        font-weight: bold;
    }
    .sub-header {
        font-size: 1.5rem;
        color: #1E88E5;
        margin-bottom: 1rem;
    }
    .metric-card {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 1.5rem;
        border-radius: 15px;
        color: white;
        text-align: center;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    }
    .stButton > button {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border: none;
        padding: 0.5rem 2rem;
        border-radius: 25px;
        font-weight: bold;
    }
</style>
""", unsafe_allow_html=True)

# Title
st.markdown('<h1 class="main-header">📊 Reddit Real-Time Sentiment Dashboard</h1>', unsafe_allow_html=True)
st.markdown('<p style="text-align: center; color: #666; font-size: 1.1rem;">Monitor sentiment trends across Reddit communities in real-time</p>', unsafe_allow_html=True)

# Sidebar
with st.sidebar:
    st.markdown('<h2 class="sub-header">⚙️ Dashboard Controls</h2>', unsafe_allow_html=True)

    # File selection
    data_files = [f for f in os.listdir('.') if f.endswith('.csv') and 'reddit' in f.lower()]
    selected_file = st.selectbox(
        "📁 Select Data File",
        data_files,
        index=0 if data_files else None
    )

    # Auto-refresh toggle
    auto_refresh = st.toggle("🔄 Auto-refresh every 30 seconds", value=False)

    # Sentiment analysis options
    st.markdown("### 📊 Analysis Settings")
    analyze_sentiment = st.checkbox("Perform sentiment analysis", value=True)

    # Date range filter
    st.markdown("### 📅 Date Range")
    date_range = st.date_input(
        "Select date range",
        value=[datetime.now() - timedelta(days=7), datetime.now()],
        max_value=datetime.now()
    )

    # Subreddit filter
    st.markdown("### 🌐 Subreddit Filter")
    all_subreddits = st.checkbox("Select all subreddits", value=True)

    # Refresh button
    if st.button("🔄 Refresh Data", use_container_width=True):
        st.rerun()

# Data loading function
@st.cache_data(ttl=30 if auto_refresh else 300)
def load_data(file_path):
    """Load and process data"""
    if not os.path.exists(file_path):
        return pd.DataFrame()

    try:
        df = pd.read_csv(file_path)

        # Convert timestamp columns
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
        if 'collected_at' in df.columns:
            df['collected_at'] = pd.to_datetime(df['collected_at'])

        # Add sentiment if not present
        if 'title_sentiment' not in df.columns and analyze_sentiment:
            df['title_sentiment'] = df['title'].apply(
                lambda x: TextBlob(str(x)).sentiment.polarity
            )
            # Categorize sentiment
            df['sentiment_category'] = df['title_sentiment'].apply(
                lambda x: 'positive' if x > 0.1 else ('negative' if x < -0.1 else 'neutral')
            )

        return df
    except Exception as e:
        st.error(f"Error loading data: {str(e)}")
        return pd.DataFrame()

# Main dashboard logic
if selected_file:
    df = load_data(selected_file)

    if not df.empty:
        # Apply filters
        if len(date_range) == 2:
            mask = (df['timestamp'].dt.date >= date_range[0]) & (df['timestamp'].dt.date <= date_range[1])
            df = df[mask]

        if not all_subreddits and 'subreddit' in df.columns:
            selected_subs = st.multiselect(
                "Select specific subreddits",
                df['subreddit'].unique(),
                default=df['subreddit'].unique()[:3]
            )
            if selected_subs:
                df = df[df['subreddit'].isin(selected_subs)]

        # Metrics row
        col1, col2, col3, col4 = st.columns(4)

        with col1:
            st.markdown('<div class="metric-card">', unsafe_allow_html=True)
            st.metric("Total Posts", len(df), delta=f"{len(df):,}")
            st.markdown('</div>', unsafe_allow_html=True)

        with col2:
            if 'upvotes' in df.columns:
                avg_upvotes = df['upvotes'].mean()
                st.markdown('<div class="metric-card">', unsafe_allow_html=True)
                st.metric("Avg Upvotes", f"{avg_upvotes:.0f}")
                st.markdown('</div>', unsafe_allow_html=True)

        with col3:
            if 'num_comments' in df.columns:
                avg_comments = df['num_comments'].mean()
                st.markdown('<div class="metric-card">', unsafe_allow_html=True)
                st.metric("Avg Comments", f"{avg_comments:.0f}")
                st.markdown('</div>', unsafe_allow_html=True)

        with col4:
            if 'sentiment_category' in df.columns:
                sentiment_score = df[df['sentiment_category'] == 'positive'].shape[0] - df[df['sentiment_category'] == 'negative'].shape[0]
                st.markdown('<div class="metric-card">', unsafe_allow_html=True)
                st.metric("Sentiment Score", sentiment_score)
                st.markdown('</div>', unsafe_allow_html=True)

        # Tabs for different visualizations
        tab1, tab2, tab3, tab4 = st.tabs(["📈 Sentiment Analysis", "📊 Subreddit Stats", "🔥 Top Content", "📋 Raw Data"])

        with tab1:
            if 'sentiment_category' in df.columns:
                col1, col2 = st.columns(2)

                with col1:
                    # Sentiment distribution pie chart
                    sentiment_counts = df['sentiment_category'].value_counts()
                    fig1 = px.pie(
                        values=sentiment_counts.values,
                        names=sentiment_counts.index,
                        title="Sentiment Distribution",
                        color=sentiment_counts.index,
                        color_discrete_map={
                            'positive': '#00CC96',
                            'negative': '#EF553B',
                            'neutral': '#636EFA'
                        },
                        hole=0.3
                    )
                    fig1.update_traces(textposition='inside', textinfo='percent+label')
                    st.plotly_chart(fig1, use_container_width=True)

                with col2:
                    # Sentiment over time
                    if len(df) > 10:
                        sentiment_time = df.groupby([df['timestamp'].dt.date, 'sentiment_category']).size().reset_index(name='count')
                        fig2 = px.line(
                            sentiment_time,
                            x='timestamp',
                            y='count',
                            color='sentiment_category',
                            title="Sentiment Trends Over Time",
                            markers=True,
                            color_discrete_map={
                                'positive': '#00CC96',
                                'negative': '#EF553B',
                                'neutral': '#636EFA'
                            }
                        )
                        st.plotly_chart(fig2, use_container_width=True)

        with tab2:
            if 'subreddit' in df.columns:
                col1, col2 = st.columns(2)

                with col1:
                    # Posts by subreddit
                    subreddit_counts = df['subreddit'].value_counts().head(10)
                    fig3 = px.bar(
                        x=subreddit_counts.index,
                        y=subreddit_counts.values,
                        title="Top Subreddits by Post Count",
                        labels={'x': 'Subreddit', 'y': 'Post Count'},
                        color=subreddit_counts.values,
                        color_continuous_scale='Viridis'
                    )
                    st.plotly_chart(fig3, use_container_width=True)

                with col2:
                    # Engagement by subreddit
                    if 'upvotes' in df.columns:
                        engagement = df.groupby('subreddit').agg({
                            'upvotes': 'mean',
                            'num_comments': 'mean'
                        }).round(1).head(10)

                        fig4 = go.Figure(data=[
                            go.Bar(name='Avg Upvotes', x=engagement.index, y=engagement['upvotes']),
                            go.Bar(name='Avg Comments', x=engagement.index, y=engagement['num_comments'])
                        ])
                        fig4.update_layout(
                            title="Engagement by Subreddit",
                            barmode='group',
                            xaxis_title="Subreddit",
                            yaxis_title="Count"
                        )
                        st.plotly_chart(fig4, use_container_width=True)

        with tab3:
            # Top posts table
            st.subheader("🔥 Top Performing Posts")

            if 'upvotes' in df.columns:
                top_posts = df.nlargest(15, 'upvotes')[['title', 'subreddit', 'upvotes', 'num_comments', 'timestamp']]
                top_posts['timestamp'] = top_posts['timestamp'].dt.strftime('%Y-%m-%d %H:%M')

                # Format as HTML table with styling
                st.dataframe(
                    top_posts.style.background_gradient(subset=['upvotes'], cmap='Greens'),
                    use_container_width=True,
                    height=400
                )

        with tab4:
            # Raw data explorer
            st.subheader("📋 Data Explorer")

            # Search functionality
            search_term = st.text_input("🔍 Search in titles:")
            if search_term:
                filtered_df = df[df['title'].str.contains(search_term, case=False, na=False)]
            else:
                filtered_df = df

            # Column selector
            available_columns = filtered_df.columns.tolist()
            selected_columns = st.multiselect(
                "Select columns to display",
                available_columns,
                default=available_columns[:min(8, len(available_columns))]
            )

            if selected_columns:
                st.dataframe(filtered_df[selected_columns], use_container_width=True, height=400)

            # Data summary
            with st.expander("📊 Data Summary"):
                col1, col2 = st.columns(2)
                with col1:
                    st.write("**Data Shape:**", df.shape)
                    st.write("**Time Range:**", df['timestamp'].min(), "to", df['timestamp'].max())
                with col2:
                    st.write("**Missing Values:**")
                    missing = df.isnull().sum()
                    for col, count in missing[missing > 0].items():
                        st.write(f"  - {col}: {count}")

            # Download button
            csv = df.to_csv(index=False)
            st.download_button(
                label="📥 Download CSV",
                data=csv,
                file_name=f"reddit_data_export_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv",
                mime="text/csv",
                use_container_width=True
            )

        # Auto-refresh logic
        if auto_refresh:
            st.write(f"🔄 Next auto-refresh in 30 seconds...")
            time.sleep(30)
            st.rerun()

    else:
        st.warning("⚠️ No data found in the selected file. Please ensure data has been collected.")

        # Instructions for first-time setup
        with st.expander("🔧 First-time setup instructions"):
            st.write("""
            1. **Run the Reddit scraper** to collect data:
               ```bash
               python realtime_scraper.py
               ```

            2. **Wait for data collection** (at least one cycle)

            3. **Refresh this page** to see the data

            4. **Select the CSV file** from the sidebar
            """)

        if st.button("📖 View Sample Data Structure"):
            sample_data = pd.DataFrame({
                'title': ['Sample Post 1', 'Sample Post 2'],
                'subreddit': ['AskReddit', 'technology'],
                'upvotes': [100, 250],
                'num_comments': [25, 50],
                'timestamp': [datetime.now(), datetime.now() - timedelta(hours=1)],
                'sentiment_category': ['positive', 'neutral']
            })
            st.dataframe(sample_data)
else:
    st.error("❌ No data files found. Please run the Reddit scraper first.")

# Footer
st.markdown("---")
col1, col2, col3 = st.columns([1, 2, 1])
with col2:
    st.markdown("""
    <div style="text-align: center; color: #666;">
    <p><strong>🚀 Reddit Sentiment Analysis Dashboard</strong></p>
    <p>Powered by Streamlit, Plotly & Reddit API</p>
    <p>Last updated: {}</p>
    </div>
    """.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S")), unsafe_allow_html=True)

2025-12-02 21:35:28.680 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 21:35:28.682 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 21:35:28.804 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-12-02 21:35:28.805 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 21:35:28.806 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 21:35:28.809 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-02 21:35:28.811 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn